이 장에서는 확률에 기반한 전통적인 Lanuage Model에 대해서 배웁니다. 확률에 기반한 방법은 우리가 실제 사용하는 자연어를 근사하기에는 많은 한계가 있었고, 요즘 들어 딥 러닝이 그러한 한계를 많이 해결해주면서 우리가 배우게 될 확률에 기반한 Language Model 대신 새로운 대안으로 떠올랐습니다.

하지만 그럼에도 여전히 딥 러닝을 사용하지 않더라도, 확률에 기반한 Language Model로도 해결할 수 있는 문제들이 많이 남아있습니다. 또한, 이 챕터에서 이해한 수학적 수식들이 향후 딥 러닝 모델을 공부할 때도 많은 도움이 될 것입니다.

# 1. 직관

가령, 비행기를 타려고 공항에 갔는데 지각을 하는 바람에 비행기를 [?] 라는 문장이 있습니다. '비행기를' 다음에 어떤 단어가 오게될지, 우리는 손쉽게 '놓쳤다' 라고 예상할 수 있습니다. 우리는 우리의 지식에 기반하여, 나올 수 있는 여러 단어들을 비교해본 결과 놓쳤다라는 단어가 나올 확률이 가장 높다고 판단한 것입니다.

그렇다면 컴퓨터에게 위의 문장을 주고나서, '비행기를' 다음에 나올 단어를 예측해보라고 한다면 과연 어떻게 최대한 정확히 예측할 수 있을까요? 그 때 사용하는 것이 바로 언어 모델이라는 것입니다.

**Input : a Sentence**
**Output : the probability of the input sentence**

문장이 주어졌을 때, 이 문장의 확률을 알려주는 것입니다. 즉, 다른 말로는 문장에 Scoring(점수)를 주는 것이라고도 볼 수 있습니다. 이는 Text Classification 문제와는 달리 비지도 학습입니다.(하지만, 순서가 있는 지도학습으로 문제를 바꿔서 풀 수 있습니다. 뒤에 자세히 설명할 것입니다.)

Language Model이 Language Generation과 어째서 연관이 있다고 볼 수 있는 것일까요? 만약 여러분들이 Sentence에 대해서 Scoring을 굉장히 잘하는 Language Model을 갖고 있다고 합시다. Generation 문제에서 모든 가능한 Sentence에 대해서 Scoring이 되어 있다면, 제일 Score가 높은 Sentence를 뽑으면 그것이 최적의 Generation이 되지 않을까요?

그런데, 모든 가능한 Sentence에 대해서 Scoring을 한다는 것은 다소 비현실적인 이야기처럼 들립니다. 그렇다면 모든 가능한 Sentence에 대해서 Scoring을 하지말고, 앞에 나온 몇 단어만을 기준으로 Partial Scoring을 하겠다는 것입니다.

가령, 문장에서 앞 몇 단어를 봤더니 이미 Partial Score가 낮다면, 이 문장을 후보에서 뺄 수 있겠죠. 여기서는 당장 구체적으로 이해가 가지 않아도 좋습니다. 뒤에서 더 자세히 다룰 것입니다. 즉, 하고자 하는 말은 문장에 확률을 부여하는 것은 Generation 문제와 연관이 있다는 것입니다.



# 2. 사용 예시

앞서 언어 모델이 문장에 확률을 계산는 일이라고 언급했습니다. 그럼 문장의 확률을 계산하는 것이 왜 필요한지 예를 들어보도록 하겠습니다. 물론, 여기서 대문자 P는 확률을 의미합니다.

1. 기계 번역(Machine Translation)

    - P(나는 버스를 탔다) > P(나는 버스를 태운다): 언어 모델은 두 문장을 비교하여 좌측의 문장의 확률이 더 높다고 판단합니다.

2. 오타 교정(Spell Correction)

    - 선생님이 교실로 부리나케 P(달려갔다) > P(잘려갔다): 언어 모델은 두 문장을 비교하여 좌측의 문장의 확률이 더 높다고 판단합니다.

3. 음성 인식(Speech Recognition)

    - P(나는 메롱을 먹는다) < P(나는 메론을 먹는다): 언어 모델은 두 문장을 비교하여 우측의 문장의 확률이 더 높다고 판단합니다.

언어 모델은 위와 같이 보다 적절한 문장을 선택하는 일에 확률을 사용하는 것입니다.



# 3. 확률 언어 모델

이 장을 이해하기 위해서는 조건부 확률에 대한 내용을 상기할 필요가 있습니다. 조건부 확률은 두 확률 P(A), P(B)에 대해서 아래와 같은 관계를 갖습니다.

p(B|A) = P(A,B)/P(A)
P(A,B) = P(A)P(B|A)

더 많은 확률에 대해서 일반화해봅시다. 4개의 확률이 조건부 확률의 관계를 가질 때, 아래와 같이 표현할 수 있습니다.
P(A,B,C,D) = P(A)P(B|A)P(C|A,B)P(D|A,B,C)




P(An, adorable, little, boy, is, spreading, smiles)를 계산하고자 합니다.
각 단어는 문맥이라는 관계로 인해 이전 단어의 영향을 받아 나온 단어입니다. 그렇기 때문에 우리는 이 문장 전체의 확률을 구하고자 조건부 확률을 사용하게 됩니다. 앞서 언급한 조건부 확률의 일반화 식을 문장의 확률 관점에서 다시 적어보면, 문장의 확률은 문장 내 단어의 joint probability로 구성되는 것입니다.



즉, 실제 예제에 해당 식을 적용해보면 다음과 같습니다.

P(“An adorable little boy is spreading smiles”) =
P(An) × P(adorable|An) × P(little|An adorable)
× P(is|An adorable little) × P(spreading|An adorable little is)
× P(smiles|An adorable little is spreading)

전체 문장의 확률을 계산하기 위해서, 다음 단어가 올 확률들을 곱하는 것을 볼 수 있습니다.



**실제로는 count 기반으로 접근한다.**

문장의 확률을 계산하기 위해서 조건부 확률식을 사용한다는 것은 알았습니다.
조건부 확률식을 사용하는 것은 좋은데, 애초에 P라는 확률을 계산하는 방법이 있어야 하지 않을까요? 그렇다면, 실제 application에서 대체 어떻게 확률을 계산할까요? 정답은 count에 기반하여 확률을 계산합니다.

An adorable little boy가 나왔을 때, is가 나올 확률을 계산하고자 합니다. 그 확률은 위와 같습니다. 예를 들어, 갖고 있는 데이터를 기준으로 An adorable little boy가 100번 언급됐는데 그 다음에 is가 나온 것이 30개라고 한다면, P(is|An adorable little boy)는 30%로 계산됩니다.


**count 기반의 접근은 사실 한계가 있다.**

지금까지 설명한 것을 바탕으로 언어 모델에 대해서 다시 재정의 해보자면, 우리가 사용하는 말의 정확한 확률 분포를 근사 모델링하는 것이 언어 모델이라고 볼 수 있습니다.

이것이 무슨 의미냐면, 실제로 우리가 정확하게 알아볼 방법은 없겠지만 현실 자연어에서도 An adorable little boy가 나왔을 때 is가 나올 확률이라는 것이 존재할 것입니다. 이를 우리가 사용하는 말의 정확한 확률 분포, 현실에서의 확률 분포라고 명칭해볼 수 있습니다.

그리고 우리는 여러 코퍼스를 통해 훈련하여, LM을 통해 현실에서의 확률 모델을 근사(approximation)하는 것이 우리의 목표인 것입니다. 그런데 count 기반으로 접근하려고 한다면, 우리가 갖고있는 코퍼스에는 엄청난 양의 데이터가 있어야 합니다.



가령, 우리가 An adorable little boy가 나왔을 때 is가 나올 확률을 계산하고 싶었지만, 우리가 갖고있는 코퍼스에 An adorable little boy is라는 문장 자체가 없었다고 한다면 우리가 갖고있는 확률을 계산할 수 없습니다. 분자 또는 분모가 0이 되어버리는 현상이 발생하는 것입니다. 그런데 우리가 이 확률을 0이라고 할 수 있을까요? 아닙니다. 현실에선 An adorable little boy is 라는 단어의 나열이 분명히 존재하고 있기 때문입니다.

위의 문제를 완전히 해결할 수는 없지만, 그래도 조금은 일반화(generalization)하는 방법이 있습니다. 바로 마르코프의 가정과 그 이론을 이용한 n-gram입니다. (물론, n-gram도 여전히 한계를 가지며 그 대안이 되는 것이 딥러닝입니다. 이해의 흐름을 위해서 미리 언급합니다.)



# 4. 한글에서의 언어 모델

지금까지 확률을 바탕으로 한 언어 모델에 대해서 배워보았습니다. 그런데 영어나 기타 언어에 비해서 한글은 언어 모델로 확률을 예측하기가 훨씬 까다롭습니다. 사실, 대부분은 이미 토큰화에서 언급했던 내용이기도 한데, 적절한 토큰화가 언어 모델에서 필수적이기 때문입니다.

1. 한글은 어순이 중요하지 않다.
한글에서는 어순이 중요하지 않습니다. 그래서 이전 단어가 주어졌을때, 다음 단어가 나타날 확률을 구해야하는데 어순이 중요하지 않다는 것은 어떤 단어든 나타나도 된다는 의미입니다.

예를 들어보도록 합시다.

Ex)
① 나는 점심을 먹습니다 식당에서.
② 나는 식당에서 점심을 먹습니다.
③ 식당에서 점심을 먹습니다.
④ 나는 점심을 식당에서 먹습니다.

4개의 문장은 전부 의미가 통하는 것을 볼 수 있습니다. 심지어 '나는' 이라는 주어를 생략해도 말이 되버립니다. 이렇게 단어 순서를 뒤죽박죽으로 바꾸어놔도 한글은 의미가 전달 되기 때문에 확률에 기반한 언어 모델이 제대로 다음 단어를 예측하기가 어렵습니다.

2. 다양한 조사가 나올 수 있다.
또 다른 이유로는 조사가 있습니다. 조사때문에 문장에서 발생가능한, 또는 예측해야할 단어의 수가 엄청나게 늘어나버립니다. 영어는 기본적으로 조사가 없습니다. 하지만 한글에는 어떤 행동을 하는 동사의 주어나 목적어를 위해서 조사라는 것이 있습니다.

가령 '그녀'라는 단어 하나만 해도 그녀가, 그녀를, 그녀의, 그녀와, 그녀로, 그녀께서, 그녀처럼 등과 같이 다양한 경우가 존재합니다. 그렇기 때문에, 한글에서는 토큰화를 통해 접사나 조사를 분리하는 것은 중요한 작업이 되기도 합니다.

3. 한글은 띄어쓰기가 제대로 되지 않습니다.
한글은 띄어쓰기를 제대로 하지 않아도 의미가 전달되며, 띄어쓰기 규칙 또한 상대적으로 까다로운 언어이기 때문에 여러분들이 자연어 처리를 하는 것에 있어서 한글 코퍼스는 띄어쓰기가 제대로 지켜지지 않는 경우가 많습니다. 이 경우에 토큰이 제대로 분리 되지 않아 언어 모델은 제대로 동작하지 않습니다.

